In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from utils import RandomSeedResetter
from utils.ReservoirComputing import EchoStateNetwork

RandomSeedResetter.reset_random_seed(0)

In [ ]:
input_train_shuffle = np.load('data/lorenz/input_train_shuffle.npy')
output_train_shuffle = np.load('data/lorenz/output_train_shuffle.npy')
input_valid = np.load('data/lorenz/input_valid.npy')
output_valid = np.load('data/lorenz/output_valid.npy')
input_test = np.load('data/lorenz/input_test.npy')
output_test = np.load('data/lorenz/output_test.npy')

In [ ]:
DIM_INPUT = len(input_train_shuffle[0, 0, :])
LEN_TIME = len(input_train_shuffle[0, :])
SIZE_RESERVOIR = 960
DIM_INTER = SIZE_RESERVOIR
LEAK_RATE = [1, 0.9, 0.8]
SIZE_LEAK = len(LEAK_RATE[:])
PROB_INPUT = 1
PROB_RESERVOIR = 1

In [ ]:
wide_esn = [[], [], []]

In [ ]:
for idx_leak in range(SIZE_LEAK):
    esn = EchoStateNetwork(
                    SIZE_INPUT=DIM_INPUT,
                    SIZE_RESERVOIR=SIZE_RESERVOIR,
                    PROB_INPUT=PROB_INPUT,
                    PROB_RESERVOIR=PROB_RESERVOIR,
                    LEAK_RATE=LEAK_RATE[idx_leak])
    wide_esn[idx_leak].append(esn)

In [ ]:
time_init = time.time()
SIZE_TRAIN = len(input_train_shuffle[:])
input_esn = np.zeros((DIM_INPUT, 1))
log_wide_esn_train_shuffle = np.zeros((SIZE_LEAK, SIZE_TRAIN, SIZE_RESERVOIR))

In [ ]:
for idx_batch in range(SIZE_TRAIN):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):
        input_esn[:, 0] = input_train_shuffle[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            wide_esn[idx_leak][0].activate(input_esn[:, :])

    for idx_leak in range(SIZE_LEAK):
        log_wide_esn_train_shuffle \
            [idx_leak, idx_batch, :]  \
            = wide_esn[idx_leak][0].state_reservoir[:]
        # reset reservoir state among time steps
        wide_esn[idx_leak][0].reset_reservoir_state()

In [ ]:
print('End calculating of the train')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/wide_esn_lorenz_960/log_wide_esn_train_shuffle.npy', log_wide_esn_train_shuffle)

In [ ]:
time_init = time.time()
SIZE_VALID = len(input_valid[:])
log_wide_esn_valid = np.zeros((SIZE_LEAK, SIZE_VALID, SIZE_RESERVOIR*DEPTH_ESN))

In [ ]:
for idx_batch in range(SIZE_VALID):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):
        input_esn[:, 0] = input_valid[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            wide_esn[idx_leak][0].activate(input_esn[:, :])

    for idx_leak in range(SIZE_LEAK):
        log_wide_esn_train_shuffle \
            [idx_leak, idx_batch, :]  \
            = wide_esn[idx_leak][0].state_reservoir[:]
        # reset reservoir state among time steps
        wide_esn[idx_leak][0].reset_reservoir_state()

In [ ]:
print('End calculating of the validation')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/wide_esn_lorenz_960/log_wide_esn_valid.npy', log_wide_esn_valid)

In [ ]:
time_init = time.time()
SIZE_TEST = len(input_test[:])
log_deep_esn_test = np.zeros((SIZE_LEAK, SIZE_TEST, SIZE_RESERVOIR*DEPTH_ESN))

In [ ]:
for idx_batch in range(SIZE_TEST):
    if idx_batch%500 == 0:
        print('idx_batch:', idx_batch)
        time_end = time.time()
        print('{:.1f}'.format((time_end-time_init)/60), 'min')

    for idx_time in range(LEN_TIME):
        input_esn[:, 0] = input_test[idx_batch, idx_time, :]
        for idx_leak in range(SIZE_LEAK):
            wide_esn[idx_leak][0].activate(input_esn[:, :])

    for idx_leak in range(SIZE_LEAK):
        log_wide_esn_train_shuffle \
            [idx_leak, idx_batch, :]  \
            = wide_esn[idx_leak][0].state_reservoir[:]
        # reset reservoir state among time steps
        wide_esn[idx_leak][0].reset_reservoir_state()

In [ ]:
print('End calculating of the test')
time_end = time.time()
print('{:.1f}'.format((time_end-time_init)/60), 'min')

In [ ]:
np.save('data/wide_esn_lorenz_960/log_wide_esn_test.npy', log_wide_esn_test)